In [179]:
from pydantic import BaseModel
from typing import List, Optional
import os
import json
from itertools import product
from copy import deepcopy
import random
import pandas as pd

class Player(BaseModel):
    name: str
    buchholz_score: int
    points_for: int
    points_against: int
    wins: int
    losses: int
    had_bye: bool = False
    

class Game(BaseModel):
    player1: str
    player2: Optional[str]
    points1: int = 0
    points2: int = 0


# Prediction, what games could possibly happen in the next round, and how likely they are
class GamePrediction(BaseModel):
    player1: str
    player2: Optional[str]
    confidence: float  # 0..1

class Round(BaseModel):
    number: int
    games: List[Game]
    preset: bool = False

class Tournament(BaseModel):
    name: str
    folder: str
    rounds: List[Round]
    players: List[Player]

    def __init__(self, name: str,  players: List[str]):
        name = name
        folder = os.path.join("data", name)
        os.makedirs(folder, exist_ok=True)

        super().__init__(rounds=[], players=[], folder=folder, name=name)

        for p in players:
            self.players.append(Player(
                name=p,
                buchholz_score=0,
                points_for=0,
                points_against=0,
                wins=0,
                losses=0,
            ))

        self.save_state()

    def __init__(self, **data):
        super().__init__(**data)
        self.load_rounds()


    def save_state(self):
        data = self.model_dump()
        data["rounds"] = []  # prevent duplication

        with open(os.path.join(self.folder, "tournament.json"), "w") as f:
            json.dump(data, f, indent=2)

    def all_games(self):
        return [g for r in self.rounds for g in r.games]



    @classmethod
    def load(cls, name: str):
        with open(os.path.join("data", name, "tournament.json")) as f:
            return cls.model_validate_json(f.read())
        
    def load_rounds(self):
        self.rounds = []
        i = 1
        while True:
            path = self.round_path(i)
            if not os.path.exists(path):
                break
            with open(path) as f:
                r = Round.model_validate_json(f.read())
                self.rounds.append(r)
            i += 1
    
    def load_round(self, number: int) -> Optional[Round]:
        path = self.round_path(number)

        if not os.path.exists(path):
            return None

        with open(path) as f:
            return Round.model_validate_json(f.read())

    def all_rounds(self) -> List[Round]:
        rounds = []
        i = 1

        while True:
            r = self.load_round(i)
            if not r:
                break
            rounds.append(r)
            i += 1

        return rounds

    def save_round(self, round: Round):
        # Replace if round with same number exists
        self.rounds = [r for r in self.rounds if r.number != round.number]
        self.rounds.append(round)

        with open(self.round_path(round.number), "w") as f:
            f.write(round.model_dump_json(indent=2))

    def round_path(self, number: int):
        return os.path.join(self.folder, f"round_{number:03}.json")
    
    def reset_player_points_and_records(self) -> int:
        for player in self.players:
            player.points_for = 0
            player.points_against = 0
            player.wins = 0
            player.losses = 0
            player.buchholz_score = 0

    def calculate_points_and_win_loss(self, games: List[Game]):
        # reset first
        self.reset_player_points_and_records()

        for game in games:
            p1 = next(p for p in self.players if p.name == game.player1)
            if game.player2:
                p2 = next(p for p in self.players if p.name == game.player2)
            else:
                # BYE counts as a win
                p1.wins += 1
                continue

            if game.points1 > game.points2:
                p1.wins += 1
                p2.losses += 1
            elif game.points1 < game.points2:
                p2.wins += 1
                p1.losses += 1

            p1.points_for += game.points1
            p1.points_against += game.points2
            p2.points_for += game.points2
            p2.points_against += game.points1

        self.calculate_buchholz_scores()



    def calculate_buchholz_scores(self) -> int:
        for player in self.players:
            score = 0
            for game in self.all_games():
                if game.player1 == player.name:
                    opponent = game.player2
                elif game.player2 == player.name:
                    opponent = game.player1
                else:
                    continue

                opponent = self.find_player_by_name(opponent)
                if opponent is None:
                    continue
                score += opponent.wins
            player.buchholz_score = score

    def have_played(self, p1: Player, p2: Player) -> bool:
        for game in self.all_games():
            if (
                (game.player1 == p1.name and game.player2 == p2.name)
                or
                (game.player1 == p2.name and game.player2 == p1.name)
            ):
                return True
        return False
    

    def find_player_by_name(self, name: str) -> Optional[Player]:
        for player in self.players:
            if player.name == name:
                return player
        return None
    

    def is_last_round_fully_played(self, rounds: List[Round]) -> bool:
        if not rounds:
            return True
        last_round = rounds[-1]
        for game in last_round.games:
            if game.points1 == 0 and game.points2 == 0:
                return False
        return True

    def create_and_show_next_round_pairings(self) -> List[Game]:

        
        rounds = self.all_rounds()
        already_paired = []

        # check if last round has no full list of games (preset round)
        if rounds and rounds[-1].preset:
            already_paired = rounds[-1].games
            rounds = rounds[:-1]

        # ---------- Safety check: last round fully played ----------
        if not self.is_last_round_fully_played(rounds):
            raise ValueError(f"Cannot create next round: Round {rounds[-1].number} is not fully played.")

        self.calculate_points_and_win_loss(self.all_games())

        games = self.generate_next_round_pairings(already_paired)

        round_number = max((r.number for r in rounds), default=0) + 1
        round_obj = Round(number=round_number, games=games)
        self.save_round(round_obj)

        return games
    

    def generate_next_round_pairings(self, pre_paired_games: List[Game] = []) -> List[Game]:
        # sort players by wins and buchholz score and points against asc
        self.players.sort(
            key=lambda p: (p.wins, p.buchholz_score, -p.points_against),
            reverse=True,
        )

        unpaired = self.players[:]
        # remove already paired players
        unpaired = [p for p in unpaired if all(p.name != g.player1 and p.name != g.player2 for g in pre_paired_games)]
        games = pre_paired_games[:]


        # ---------- BYE ----------
        if len(unpaired) % 2 == 1:
            for player in reversed(unpaired):
                if not player.had_bye:
                    bye_player = player
                    break
            else:
                bye_player = unpaired[-1]

            unpaired.remove(bye_player)
            bye_player.had_bye = True

            games.append(Game(
                player1=bye_player.name,
                player2=None,
                points1=1,
                points2=0
            ))

        # ---------- pairing ----------
        while len(unpaired) > 1:
            p1 = unpaired.pop(0)

            opponent_index = None
            for i, p2 in enumerate(unpaired):
                if not self.have_played(p1, p2):
                    opponent_index = i
                    break

            if opponent_index is None:
                opponent_index = 0

            p2 = unpaired.pop(opponent_index)

            games.append(Game(
                player1=p1.name,
                player2=p2.name
            ))

        return games
    

    def get_standings(self) -> List[Player]:
        self.load_rounds()


        # make sure only full rounds are counted for standings
        rounds = self.all_rounds()
        if not self.is_last_round_fully_played(rounds):
            rounds = rounds[:-1]
        self.calculate_points_and_win_loss([g for r in rounds for g in r.games])

        return sorted(
            self.players,
            key=lambda p: (p.wins, p.buchholz_score),
            reverse=True,
        )
    

    

    def predict_next_round(self, points_for_win: int = 4, top_n: int = 20) -> List[GamePrediction]:
        """
        Predict next round games based on all possible outcomes of unfinished games in the last round.
        """
        rounds = self.all_rounds()
        if not rounds:
            raise ValueError("No rounds yet — cannot predict next round.")

        last_round = rounds[-1]

        # Identify unfinished games (points1 == points2 == 0)
        unfinished_games = [g for g in last_round.games if g.points1 == 0 and g.points2 == 0]
        if not unfinished_games:
            raise ValueError("Last round is fully played — no prediction needed.")
        if len(unfinished_games) > 2:
            raise ValueError("Too many unfinished games to predict reliably.")

        # Generate all possible outcomes for unfinished games
        # Each game has 3 possibilities: player1 win, player2 win, draw (optional)
        outcome_options = []
        for g in unfinished_games:
            game_outcomes=[]
            if g.player2 is None:
                # BYE is always a win
                game_outcomes.append((g.player1, None, 1, 0))
            else:
                # append loser scoring up to points_for_win - 1 don't allow for draws
                for loser_points in range(points_for_win):
                    game_outcomes.append((g.player1, g.player2, points_for_win, loser_points))  # p1 wins
                    game_outcomes.append((g.player1, g.player2, loser_points, points_for_win))  # p2 wins
            outcome_options.append(game_outcomes)

        all_scenarios = list(product(*outcome_options))
        pairing_counts = {}

        for scenario in all_scenarios:
            # Create a deep copy of tournament to simulate
            t_copy = deepcopy(self)

            # Apply scenario results
            for p1_name, p2_name, points1, points2 in scenario:
                game = next(g for g in t_copy.rounds[-1].games if g.player1 == p1_name and g.player2 == p2_name)
                game.points1 = points1
                game.points2 = points2

            # Recalculate standings
            t_copy.calculate_points_and_win_loss(t_copy.all_games())
            pairings = t_copy.generate_next_round_pairings()

            # Count frequency
            for game in pairings:
                # order players alphabetically to avoid counting (A vs B) and (B vs A) separately
                if game.player2 and game.player1 > game.player2:
                    game.player1, game.player2 = game.player2, game.player1
                key = (game.player1, game.player2)
                pairing_counts[key] = pairing_counts.get(key, 0) + 1

        # Convert counts to confidence
        total_scenarios = len(list(product(*outcome_options)))
        predictions = [
            GamePrediction(player1=k[0], player2=k[1], confidence=v / total_scenarios)
            for k, v in pairing_counts.items()
        ]

        # Return top_n predictions
        predictions.sort(key=lambda x: x.confidence, reverse=True)
        return predictions[:top_n]


In [166]:
rounds = tourn.all_rounds()
points_for_win = 4
if not rounds:
    raise ValueError("No rounds yet — cannot predict next round.")

last_round = rounds[-1]

# Identify unfinished games (points1 == points2 == 0)
unfinished_games = [g for g in last_round.games if g.points1 == 0 and g.points2 == 0]
if not unfinished_games:
    raise ValueError("Last round is fully played — no prediction needed.")
if len(unfinished_games) > 2:
    raise ValueError("Too many unfinished games to predict reliably.")

# Generate all possible outcomes for unfinished games
# Each game has 3 possibilities: player1 win, player2 win, draw (optional)
outcome_options = []
for g in unfinished_games:
    game_outcomes=[]
    if g.player2 is None:
        # BYE is always a win
        game_outcomes.append((g.player1, None, 1, 0))
    else:
        # append loser scoring up to points_for_win - 1 don't allow for draws
        for loser_points in range(points_for_win):
            game_outcomes.append((g.player1, g.player2, points_for_win, loser_points))  # p1 wins
            game_outcomes.append((g.player1, g.player2, loser_points, points_for_win))  # p2 wins
    outcome_options.append(game_outcomes)

all_scenarios = product(*outcome_options)

In [167]:
list(all_scenarios)

[(('Sandro H.', 'Flo W.', 4, 0), ('Sevi L.', 'Janine L.', 4, 0)),
 (('Sandro H.', 'Flo W.', 4, 0), ('Sevi L.', 'Janine L.', 0, 4)),
 (('Sandro H.', 'Flo W.', 4, 0), ('Sevi L.', 'Janine L.', 4, 1)),
 (('Sandro H.', 'Flo W.', 4, 0), ('Sevi L.', 'Janine L.', 1, 4)),
 (('Sandro H.', 'Flo W.', 4, 0), ('Sevi L.', 'Janine L.', 4, 2)),
 (('Sandro H.', 'Flo W.', 4, 0), ('Sevi L.', 'Janine L.', 2, 4)),
 (('Sandro H.', 'Flo W.', 4, 0), ('Sevi L.', 'Janine L.', 4, 3)),
 (('Sandro H.', 'Flo W.', 4, 0), ('Sevi L.', 'Janine L.', 3, 4)),
 (('Sandro H.', 'Flo W.', 0, 4), ('Sevi L.', 'Janine L.', 4, 0)),
 (('Sandro H.', 'Flo W.', 0, 4), ('Sevi L.', 'Janine L.', 0, 4)),
 (('Sandro H.', 'Flo W.', 0, 4), ('Sevi L.', 'Janine L.', 4, 1)),
 (('Sandro H.', 'Flo W.', 0, 4), ('Sevi L.', 'Janine L.', 1, 4)),
 (('Sandro H.', 'Flo W.', 0, 4), ('Sevi L.', 'Janine L.', 4, 2)),
 (('Sandro H.', 'Flo W.', 0, 4), ('Sevi L.', 'Janine L.', 2, 4)),
 (('Sandro H.', 'Flo W.', 0, 4), ('Sevi L.', 'Janine L.', 4, 3)),
 (('Sandro

In [121]:
# get teilnehmer from teilnehmer.csv and randomize them
teilnehmer = pd.read_csv("teilnehmer.csv")["name"].tolist()
random.shuffle(teilnehmer)

In [180]:
# load "Bohlen-Open" tournament or create if not exists
tourn = Tournament.load("Bohlen-Open")

In [181]:
tourn.load_rounds()

In [182]:
tourn.get_standings()

[Player(name='Stephi C.', buchholz_score=0, points_for=0, points_against=0, wins=0, losses=0, had_bye=False),
 Player(name='Sandro J.', buchholz_score=0, points_for=0, points_against=0, wins=0, losses=0, had_bye=False),
 Player(name='Marina R.', buchholz_score=0, points_for=0, points_against=0, wins=0, losses=0, had_bye=False),
 Player(name='Rene W.', buchholz_score=0, points_for=0, points_against=0, wins=0, losses=0, had_bye=False),
 Player(name='Sarah C.', buchholz_score=0, points_for=0, points_against=0, wins=0, losses=0, had_bye=False),
 Player(name='Max N.', buchholz_score=0, points_for=0, points_against=0, wins=0, losses=0, had_bye=False),
 Player(name='Nico C.', buchholz_score=0, points_for=0, points_against=0, wins=0, losses=0, had_bye=False),
 Player(name='Marco S.', buchholz_score=0, points_for=0, points_against=0, wins=0, losses=0, had_bye=False),
 Player(name='Sven S.', buchholz_score=0, points_for=0, points_against=0, wins=0, losses=0, had_bye=False),
 Player(name='Frido P

In [183]:
tourn.create_and_show_next_round_pairings()

ValueError: Cannot create next round: Round 1 is not fully played.

In [184]:
tourn.predict_next_round()

[GamePrediction(player1='Rene W.', player2='Sarah C.', confidence=1.0),
 GamePrediction(player1='Nico C.', player2='Sven S.', confidence=1.0),
 GamePrediction(player1='Max N.', player2='Stephi C.', confidence=0.625),
 GamePrediction(player1='Frido P.', player2='Marina R.', confidence=0.5),
 GamePrediction(player1='Marco S.', player2='Max N.', confidence=0.375),
 GamePrediction(player1='Marco S.', player2='Marina R.', confidence=0.3125),
 GamePrediction(player1='Sandro H.', player2='Sevi L.', confidence=0.265625),
 GamePrediction(player1='Flo W.', player2='Janine L.', confidence=0.265625),
 GamePrediction(player1='Janine L.', player2='Sandro H.', confidence=0.265625),
 GamePrediction(player1='Flo W.', player2='Sevi L.', confidence=0.265625),
 GamePrediction(player1='Frido P.', player2='Sandro J.', confidence=0.25),
 GamePrediction(player1='Sandro J.', player2='Sevi L.', confidence=0.21875),
 GamePrediction(player1='Janine L.', player2='Sandro J.', confidence=0.21875),
 GamePrediction(pl

In [185]:
tourn.create_and_show_next_round_pairings()

[Game(player1='Stephi C.', player2='Max N.', points1=0, points2=0),
 Game(player1='Marco S.', player2='Marina R.', points1=0, points2=0),
 Game(player1='Frido P.', player2='Sandro J.', points1=0, points2=0),
 Game(player1='Rene W.', player2='Sarah C.', points1=0, points2=0),
 Game(player1='Nico C.', player2='Sven S.', points1=0, points2=0),
 Game(player1='Sandro H.', player2='Sevi L.', points1=0, points2=0),
 Game(player1='Flo W.', player2='Janine L.', points1=0, points2=0)]